#Sequence manager explanation

In [2]:
import sys
import os
%matplotlib inline

#Presentation

As presented in KidlearnStarter notebook, the Kidlearn library includes a set of algorithms that we develop to guide pedagogical activities. We introduce two different algorithms ZPDES that uses a simple graph of knowledge and RiARiT that includes a more complex student model. The details of the algorithms and the user studies are presented in the following paper: B. Clement, D. Roy, P.-Y. Oudeyer, M. Lopes, Multi-Armed Bandits for Intelligent Tutoring Systems, Journal of Educational Data Mining (JEDM), 2015 [(arXiv:1310.3174 [cs.AI])](http://arxiv.org/abs/1310.3174).

These algorithms are implemented in the ***seq_manager*** module of the library, with some others sequence managers used to evaluate our algorithms. In this notebook, we will describe how to define the activity for each algorithms and what are the parameters files used to define the the algorithms parameters. 

In [3]:
import kidlearn_lib as k_lib

# Lexical details

Vocabulary is important to us to describe our problems and objects that we will use, so before beginning our explanation, here is an explanation of the vocabulary that will be used in this notebook.

We will speak about **didactic parameters** (or **parameter**). These parameters are used to define an **activity**. For example, the difficulty level, the type of exercise, the contents of an exercise or the presentation of this content. 

These parameters can take different **parameter's values** (or **values**), for example the difficulty level can have a scale and you will choose level 1 or 2 or higher depending of the level of the learner. 

So to instantiate an **exercise**, one value will be chosen for each parameters needed in the current activity. An activity is a set of all possible exercises which can be defined with a set of parameters.

The parameters can be regroup in **group of parameter**. And these groups can be **hierarchically** organised. This mean the use of some paramameters will depend of other parameters. For example, if one parameter dertermine the type of exercise, some parameters will be used and others not, depending of the type of exercise chosen.   

#The sequence manager objects

There are 5 different sequence managers which can be used : 
- RiARiT : an algorithm based on student skill estimation
- ZPDES : an algorithm based on the success of the student
- Sequence : a predefined sequence of activities (defined by an expert)
- POMDP : an algorithm based on a probabilistic model to describe the activity learning.
- Random: a random sequence of all activities. 

## RiARiT : The Right Activity at the Right Time

### To describe the activity

RiARiT use tables to define the relation between skills and activities. 

Two ways exist to define the tables : 
- with a text-file, there is an example in **graph/N2.txt**.
- in json, there is an example in **graph/N2.json**.


#### Text file 

N2.txt example : 

**Competences : S0, S1, S2**  <-  here we define the skill used in the activities 
 (S0 could be addition skill for example)

**Parameters : Act1, Act2, Act3** <- here we define the didactics parameters used 
 (Act1 could be difficulty level)

**Nb_stay : 2, 1, 1 ** <- here we define the number of activities which have to be done for each didactic parameter, for act 1 there will be 2 activities with the same parameter value chosen, and for the other the parameter value will be chosen at each step

**1 | 0 | V1 | 0.2   0.1   0  | 0   0   0  | 0   0   1 **<-- example : first line of the table 

 The first value represents the number of the didactic parameter: 1 (the first parameter)
 The second value : 0 (1 else) say that this value is not used (here) to call another group of parameters, if the value is 1 that means another group of parameters is used to define the exercises.  
 The second value : V1 is a named gave to the parameter's value (number 1 here). To load another parameter group using a value, the value must have the same name than the group of the parameter file.  

 The next 3 numbers (0.2, 0.1, 0) represent the minimum required competence level, for each KC, to be able to solve that exercise

 The next 3 numbers (0, 0, 0) represent the activation threshold for this parameter's value depending on the learner skill level. Here, all values are null, so this parameter's value is activated from the beginning. 

 The next 3 numbers (0.2, 0.1, 0) represent the deactivation threshold for this parameter's value depending on the learner skill level. For this example, there is one value equal to 1, so this parameter's value will always be activated until the student has totally mastered the third skill. 

#### JSON File 

[N2.json]() example : 



In [1]:
N2 = {
    "competencies": ["S0","S1","S2"], # Definition of the competencies
    "parameters": ["Act1","Act2","Act3"], # Definition of the didactic parameters
    "nb_stay": [2,  1,  1], # Definition of the number of staying step
    "table": {
        "Act1": {  # For the first parameter
            "V1": {  # First value's name
                "hierarchical" : 0, # Is the value use others parameters 
                "impact": [0.2,  0.1,0], # the minimum required competence level
                "requir": [0,    0,  0], # required level 
                "deacti": [1,    1,  1]  # deactivation threshold
            }
            # next in the file N2.json
        }
    }
}

### To define algorithm paramaters

Here is an example : 

In [ ]:
{
    "name" : "RiaritHssbg", # First name of the algorithm 
    "RT": {                 # Here you define the activity paramaters
        "name":"N2",        # name of the activity description file (describe before)
        "path" : "RT/",     # path to the file
        "main_act":"N2"     # name of the main action (same that the name file here)
        },
# RiaritSsbg : name of a subpart of the algorithm, wich follow correspond
#              to the paramaters fo this particular part
    "RiaritSsbg": {         
        "levelupdate": 0.6, # RiaRiT parameters witch update the estimated level of the 
                            # student if he succeed the activity
        
# RiaritSsb : name of a subpart of the algorithm ...
        "RiaritSsb": {
            "filter1": 0.2,    # Filster1/2 are the paramaters that manage the bandits values 
            "filter2": 0.8,    # valBandit = f1 * valBandit + f2 * reward
            "uniformval": 0.05 # initialisation value for the first activated bandits
        }
    }
}


In [5]:
S = k_lib.config.seq_manager(params_file="RIARIT",directory="params_files")

##ZPDES : Zone of Proximal Development and Empirical Success

### To describe the activity

In [ ]:
{
    "act_prime" : "MAIN",         # What is the main action wich is sample first    
    "KC":["skill1","skill2"],     # Optional, to definie skill name 

    "MAIN":{                      # Description of the first/main bandit group    
        "actions":"ExType_H",     # Name of the actions
        "h":[1],                    
        "nb_stay":[4],
        "ssbg":[["M", "R", "MM", "RM"],
                ["param21", "param22", "param23"]]
        },
    
    "M":{
        "h":[1],
        "ssbg":[["N01","N02","N03","mod","mod","mod"]]
    },
    
    ...
}

### To define algorithms paramates

## Sequence : predefined by an expert

The predefined sequence can be define in a json file. The is an example in **sequence_def/example_seq.json**. The predefined sequence works with group or exercises. When a learner is working with a sequence, he will begin with the first exercise of the first group.

In [4]:
seq = {
    "activity": {

        "group0": [
            {"N2" : [0,0,0]},
            {"N2" : [0,1,0]},
            {"N2" : [0,0,1]}],
            
        "group1" : [
            {"N2" : [1,0,0]},
            {"N2" : [1,1,0]},
            {"N2" : [1,0,1]},
            {"N2" : [1,1,1]}],

        "group2" : [
            {"N2" : [2,0,0]},
            {"N2" : [2,1,0]},
            {"N2" : [2,0,1]},
            {"N2" : [2,1,1]}]
    }
}

## Random